In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchaudio
from torchaudio import datasets, transforms
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.optim as optim


In [ ]:
#Processing the data:

#transfer the nueral net onto a gpu

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device) #this shiuld print the device that is being used

In [ ]:
#class to load Noisy Speech Data

def load(file_path):
    waveform, sample_rate = torchaudio.load(uri=file_path)
    return waveform, sample_rate

transform = transforms.MelSpectrogram(sample_rate=8000, n_mels=128)


batch_size = 32

class NosiySpeech(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform 
        self.pathsList = [os.path.join(p, f) for p, _, files in os.walk(path) for f in files if f.endswith('.wav')]
        self.transforms_list = [transform(load(n)[0]) for n in self.pathsList]
    def __len__(self):
        return len(self.pathsList)
    
    def __getitem__(self, idx):
        return self.transforms_list[idx], load(self.pathsList[idx])[1]
    


noisy_train = f"\\Users\\Maisoon\\Documents\\Audio-Signal-Denoising-Project-\\noisy_trainset_28spk_wav"
noisy_test = f"\\Users\\Maisoon\\Documents\\Audio-Signal-Denoising-Project-\\noisy_testset_wav"

clean_train= f"\\Users\\Maisoon\\Documents\\Audio-Signal-Denoising-Project-\\clean_trainset_28spk_wav"
clean_test = f"\\Users\\Maisoon\\Documents\\Audio-Signal-Denoising-Project-\\clean_testset_wav"

#noisy data
nosiy_trainDataset = NosiySpeech(path=noisy_train, transform=transform)
print(nosiy_trainDataset.pathsList)
noisy_trainDataloader = DataLoader(nosiy_trainDataset, batch_size=32, shuffle=True)

nosiy_testDataset = NosiySpeech(path=noisy_test, transform=transform)
noisy_testDataloader = DataLoader(nosiy_trainDataset, batch_size=32, shuffle=True)

#clean data
clean_trainDataset = NosiySpeech(path=clean_train, transform=transform)
clean_trainDataloader = DataLoader(nosiy_trainDataset, batch_size=32, shuffle=True)

clean_testDataset = NosiySpeech(path=clean_test, transform=transform)
clean_testDataloader = DataLoader(nosiy_trainDataset, batch_size=32, shuffle=True)

filename = nosiy_trainDataset.pathsList[0]


In [ ]:
class DenoisingLinearModel(nn.Module):
    def __init__(self, input, output):
        super(DenoisingLinearModel, self).__init__()
        self.linear = nn.Linear(input, output) ##one layer to 

    def forward(self, x):
        x = self.linear(x)
        return x
    
input_var = 150
output_var = 150

ln_model = DenoisingLinearModel(input_var, output_var)

loss_fn = nn.MSELoss()
optimizer=torch.optim.SGD(ln_model, lr = 0.01)

correct = 0
total = 0

num_epochs = 10
for epoch in range(num_epochs):
    for noisy, clean in clean_trainDataloader:
        #noisy, clean = data
        output = ln_model(noisy)
        _, predicted = torch.max(output.data, 1)
        loss = loss_fn(output, clean)
        total += clean.size(0)
        correct += (predicted == clean).sum().item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
#CNN

import torch.nn as nn 
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        self.maxPool = nn.MaxPool2d(kernel_size=2)

        #self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128*5*4, 128)
        self.fc2 = nn.Linear(128, 1)

        self.dropout = nn.Dropout(0.5)

        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        #layer1
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        #layer 2
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        #layer 3 
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        #flatten
        x = torch.flatten(x, 1)

        #a fully conv layer with relu
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        #apply the dropout

        x = self.sigmoid(x)

        return x
    

net=Net().cuda()


num_epoch = 2


#instantiate the CNN model
#cnn = Net()


#Loss function and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

def one_epoch(model,dataloader,loss_fn,optimizer,device):
    for input, output in noisy_trainDataloader:
        #forward pass 
        outputs = model(input)
        loss = criterion(output, clean_trainDataloader)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"loss:{loss.item()}")
    

def train(model,dataloader,loss_ln,optimizer,device,num_epochs):
    for epoch in range(num_epoch):
        one_epoch(model,dataloader,criterion,optimizer,device)
    

In [ ]:
#Loss function and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)